In [1]:
# # ユーザー認証
# from google.colab import auth
# auth.authenticate_user()

#Google setting
from google import colab
colab.drive.mount('/content/gdrive')

#Directory setting
b_dir='./gdrive/My Drive/AI_Seminar/NLP_1/' # Setting working directory
dataset_dir=b_dir+'dataset/'

Mounted at /content/gdrive


In [2]:
%%bash

free -h

# ツールのダウンロード
apt-get update
apt-get install mecab file swig libmecab-dev mecab-ipadic-utf8
pip install mecab-python3==0.996.5
pip install transformers==2.11.0
pip install tqdm
pip install category_encoders

              total        used        free      shared  buff/cache   available
Mem:            12G        574M        9.8G        1.0M        2.4G         11G
Swap:            0B          0B          0B
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Re

In [3]:
# insall
import numpy as np
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from scipy import stats
from sklearn.model_selection import KFold
import os,urllib,glob,tarfile
from transformers import BertJapaneseTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# データの読み込み
train_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S")  # train.csv
valid_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1Mdyv3DPcb1AncxE7H739QWyPkMqcdjqI")  # dev.csv
test_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1yHKfDrs6ZAx4fizToTF7GL7ROvGk7Ry5")  # test.csv
# train_df = pd.read_csv("./dataset/train.csv")
# val_df = pd.read_csv("./dataset/dev.csv")
# test_submission = pd.read_csv("./dataset/test.csv")

train_df.head(5)

,movieName,title,text,rating
0,HERO,楽しみにしてるよ～～!!!,この前、HEROを１話から見直しました!!もぅドキドキで楽しいっ!!あたしも、西遊記見たとき...,5
1,インセプション,期待しすぎた…,DVDでの鑑賞。\nちょっと観て時間がたったので、ざっくりレビューです。\n\n根本的なアイ...,3
2,パッチギ！ LOVE&PEACE,真面目に生きている在日の人たちが可哀想,こんな変な映画が、全国上映されるからまた、在日批判が出てくるんじゃないの？,1
3,ダ・ヴィンチ・コード,いまいちの映画でした,はっきり言って期待しすぎたのが悪かったのか全く面白くなかったです。この手の映画は宗教がらみな...,2
4,踊る大捜査線 THE MOVIE 3 ヤツらを解放せよ！,残念なチャリティーマッチ,制作側の意図や事情を知らずにむやみに酷評するのははばかられますが、はっきり言ってつまらなかっ...,2


In [5]:
train_df = train_df.dropna()
valid_df = valid_df.dropna()
test_df = test_df.dropna()

In [6]:
# list_cols = ['movieName']
# # ce_ohe = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')
# ce_oe = ce.OrdinalEncoder(cols=list_cols, handle_unknown='impute')
# # train_df, valid_df, test_df = ce_ohe.fit_transform([train_df, valid_df, test_df])
# train_df = ce_oe.fit_transform(train_df)
# valid_df = ce_oe.fit_transform(valid_df)
# test_df = ce_oe.fit_transform(test_df)

In [7]:
train_df['text'] = train_df['title'].str.cat(train_df['text'])
valid_df['text'] = valid_df['title'].str.cat(valid_df['text'])
test_df['text'] = test_df['title'].str.cat(test_df['text'])

In [8]:
train_df['movieName'] = train_df['movieName'].astype(str)
valid_df['movieName'] = train_df['movieName'].astype(str)
test_df['movieName'] = train_df['movieName'].astype(str)

In [9]:
train_df['text'] = train_df['movieName'].str.cat(train_df['text'])
valid_df['text'] = valid_df['movieName'].str.cat(valid_df['text'])
test_df['text'] = test_df['movieName'].str.cat(test_df['text'])

In [10]:
# tokenizerの定義とvocabを作成
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
pad = tokenizer.vocab["[PAD]"]
# max_lengths=256
max_lengths=512

In [11]:
# 上記のinput_dataの作成方法をもとにdataset化
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels=[]):
        self.input_ids, self.segment_ids, self.attention_masks = [],[],[]
        for text in tqdm(texts):
            token = ["[CLS]"]+tokenizer.tokenize(text)[:max_lengths-2]+["[SEP]"]
            input_id = tokenizer.convert_tokens_to_ids(token)
            segment_id = [0]*max_lengths
            attention_mask = [1]*len(input_id)+[0]*(max_lengths - len(input_id))
            input_id = input_id+[pad]*(max_lengths-len(input_id))
            self.input_ids.append(input_id)
            self.segment_ids.append(segment_id)
            self.attention_masks.append(attention_mask)
        self.input_ids = np.array(self.input_ids)
        self.segment_ids = np.array(self.segment_ids)
        self.attention_masks = np.array(self.attention_masks)
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if len(self.labels):
            return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx], self.labels[idx]
        else:
            return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx]

In [12]:
# datasetとdataloaderの作成
batch_size = 8  # 8
# X, y = train_df.drop("rating").values, train_df["rating"].values-1
X, y = train_df["text"].values, train_df["rating"].values-1
# X, y = train_df["title"].values, train_df[["rating_1", "rating_2", "rating_3", "rating_4", "rating_5"]].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=100,stratify=y)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=100,stratify=y)
train_ds = ReviewDataset(texts=X_train, labels=y_train)
train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=batch_size, shuffle=True)
X_validOrg, y_validOrg = valid_df["text"].values, valid_df["rating"].values-1
X_valid, y_valid = np.concatenate([X_val, X_validOrg]), np.concatenate([y_val, y_validOrg])
# X_valid, _, y_valid, _ = train_test_split(X_valid, y_valid, test_size=0.64, random_state=100, stratify=y_valid)
valid_ds = ReviewDataset(texts=X_valid, labels=y_valid)
valid_dl = torch.utils.data.DataLoader(
    valid_ds, batch_size=batch_size, shuffle=False)

In [13]:
print(X_valid)
print(y_valid)

['ミスト後味最悪後味最悪の映画。\nラストで興ざめしました。\nどうしてこんなラストにしたのか意味不明。\nラストまでは、結構面白かったのに・・・\nラストで全て台無しにしています。'
 'ゼロ・グラビティ3Dを楽しむなら・・・正直内容自体は期待外れ。\n\nただ単に3Dを楽しむなら面白いと思います。'
 'るろうに剣心目にも止まらぬ早業ですな面白かったです。\n\n佐藤くんの剣心はどうなのかな～、と思っていましたが、緩急のついた演技がなかなかよかったです。\nマンガのイメージが強いので、実写で「ござる」「おろ？」とかの台詞はちょっと恥ずかしかったですが…。\n\n殺陣は見ごたえありましたね！\nマンガで描かれていた剣さばきがうまく映像化されていました。\n\n速すぎてわからないくらい。でもそれでいいのかも。\n観客に見せることを考慮してスピード緩めたら、物足りなさを感じると思います。\nDVDで見たときにスロー再生してみてもいいかも。\n\n今作はストーリーの序盤でしたが、剣心の過去に背負ったものや、人物関係などがきっちり描かれていて、納得できる内容でした。\n\nアクションシーンを楽しむつもりで見に行ったんですが、終盤の武井咲の演技に思わず涙、涙、涙。\n\n続編は未定の様ですが、ぜひ続きが見たいです。\n\n「るろうに剣心」は全巻持っていたはずなんですが、行方不明。\n中古販売したんだっけかな？\nそれすら思い出せない。\nああ、寄る年波の恐ろしさ…。\nすっかりストーリーも忘れているので、読み直したいと思います。'
 ...
 'HERO楽しい！映画との妙な距離感を覚えるような、間延びを感じる\nシーンが多かったです。\nけれど、そうかと思うと、急に自分のストライクな\n笑いのツボをつかれることもあって、感情がグルグル\n回って、慌しさのようなものが残りました。\nまさにドタバタ喜劇という印象･･･\n\n一緒に鑑賞していた友人の笑い声や、隣に座っていた\n年配の男性の笑い声など、周りから聞こえてくる笑い\n声と、自分の笑い声が重なる瞬間に、何度も感動を\n覚えました。嬉しくなるというか･･･\n\nこれからはしばらく、眠りに落ちる瞬間に、ずっと\n佐藤浩市さん演じる村田さんの姿が思い出されそうです･･･'
 'インディ・ジョーンズ／クリスタル・スカルの王

In [14]:
# Bertの事前学習済みエンコーダーを使用してクラス分類器を作成

class BertClassification(nn.Module):
    def __init__(self, bert, n_classes=2):
        super(BertClassification, self).__init__()

        # BERTモジュール
        self.bert = bert  # BERTモデル

        self.cls = nn.Linear(in_features=768, out_features=n_classes)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        pooled_output = self.bert(
                  input_ids, token_type_ids, attention_mask)[1]

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        pooled_output = pooled_output.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        outputs = self.cls(pooled_output)

        return outputs

In [16]:
cuda = torch.cuda.is_available()
# 学習済みモデルをダウンロード
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertClassification(bert, n_classes=5)
# 高速化
torch.backends.cudnn.benchmark = True
if cuda:
    model.cuda()
# optimizerの設定
# optimizer = optim.Adam(model.parameters(),lr = 4e-4, betas=(0.9, 0.999))
optimizer = optim.AdamW(model.parameters(),lr = 1e-5, betas=(0.9, 0.999))
# optimizer = optim.AdamW(model.parameters(), lr = 1e-4, betas=(0.9, 0.999))
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=(len(X_train) // batch_size + 1) * 1,
#     num_training_steps=(len(X_train) // batch_size + 1) * 5
# )
max_epoch = 4
total_steps = len(X_train) * max_epoch
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps = 0, # Default value in run_glue.py
    num_training_steps = total_steps)

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

In [17]:
# # 最後以外のBertLayerモジュールのパラメータを固定(今回は速さのためにしているだけ。精度を上げるなら固定せずに実行するべき)
# for param in model.bert.encoder.layer[:-1].parameters():
#     param.requires_grad = False

In [18]:
## 学習します。
epochs = max_epoch
# best_train_acc = 0.0
# best_val_acc = 0.0
best_train_mae = np.inf
best_valid_mae = np.inf
for epoch in range(epochs):
    total_loss = 0.0
    accuracy = 0.0
    mae = 0.0
    model.train()

    print("epoch {} start!".format(epoch+1))
    # train
    for iter_num, (input_ids, segment_ids, attention_masks, labels) in tqdm(enumerate(train_dl), total=len(train_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, segment_ids, attention_masks, labels =\
            input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
        outputs = model(input_ids = input_ids,
                        token_type_ids = segment_ids,
                        attention_mask = attention_masks)
        pred_proba = outputs.softmax(dim=-1)
        pred = torch.argmax(pred_proba, axis=1)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        accuracy += (pred==labels).sum().item()
        mae += torch.abs(pred - labels).sum().item()
        # accuracy += (pred.indices==labels).sum().item()
        # mae += torch.abs(pred.indices - labels).sum().item()
        #50 iterごとにlossとaccuracyを表示
        if (iter_num+1) % 50 == 0:
            now_size = batch_size*(iter_num+1)
            print("{} iter loss:{:.4f} accuracy:{:.4f} MAE:{:.4f}".format(
            iter_num+1,total_loss/(iter_num+1),accuracy/now_size,mae/now_size))
            # if best_train_acc < accuracy/now_size:
            #     torch.save(model.state_dict(), './dataset/model.hdf5')
            #     best_train_acc = accuracy/now_size
            if best_train_mae > mae/now_size:
                torch.save(model.state_dict(), dataset_dir+'model_train_glsww_allinAsItIs_4epochs.hdf5')
                best_train_mae = mae/now_size

    total_loss /= len(train_dl)
    accuracy /= len(train_ds)
    mae /= len(train_ds)

    # valididation
    valid_total_loss = 0.0
    valid_accuracy = 0.0
    valid_mae = 0.0
    model.eval()
    for input_ids, segment_ids, attention_masks, labels in tqdm(valid_dl):
        if cuda:
            input_ids, segment_ids, attention_masks, labels =\
            input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
        with torch.no_grad():
            outputs = model(input_ids = input_ids,
                        token_type_ids = segment_ids,
                        attention_mask = attention_masks)
        pred_proba = outputs.softmax(dim=-1)
        pred = torch.argmax(pred_proba, axis=1)
        # pred = torch.max(pred_proba, dim=-1)
        loss = criterion(outputs, labels)
        valid_total_loss += loss.item()
        valid_accuracy += (pred==labels).sum().item()
        valid_mae += torch.abs(pred - labels).sum().item()
        # valid_accuracy += (pred.indices==labels).sum().item()

    valid_total_loss /= len(valid_dl)
    valid_accuracy /= len(valid_ds)
    valid_mae /= len(valid_ds)
    print("epoch:{} total loss:{:.4f}, accuracy:{:.4f}, MAE:{:.4f}, valid_total loss:{:.4f}, valid_accuracy:{:.4f}, valid_MAE:{:.4f}"\
        .format(epoch+1,total_loss,accuracy,mae,valid_total_loss,valid_accuracy,valid_mae))
    # if best_val_acc < val_accuracy:
    #     torch.save(model.state_dict(), dataset_dir+'model.hdf5')
    #     best_val_acc = val_accuracy
    if best_valid_mae > valid_mae:
        torch.save(model.state_dict(), dataset_dir+'model_valid_glsww_allinAsItIs_4epochs.hdf5')
        best_valid_mae = valid_mae
# torch.save(model.state_dict(), dataset_dir+'model.hdf5')

epoch 1 start!


50 iter loss:1.6456 accuracy:0.2750 MAE:1.4175
100 iter loss:1.6183 accuracy:0.2725 MAE:1.3887
150 iter loss:1.5842 accuracy:0.2867 MAE:1.3108
200 iter loss:1.5414 accuracy:0.3019 MAE:1.2288
250 iter loss:1.4983 accuracy:0.3250 MAE:1.1530
300 iter loss:1.4698 accuracy:0.3387 MAE:1.1013
350 iter loss:1.4365 accuracy:0.3521 MAE:1.0589
400 iter loss:1.4126 accuracy:0.3644 MAE:1.0191
450 iter loss:1.3876 accuracy:0.3744 MAE:0.9864
500 iter loss:1.3699 accuracy:0.3845 MAE:0.9590
550 iter loss:1.3507 accuracy:0.3945 MAE:0.9302
600 iter loss:1.3355 accuracy:0.4035 MAE:0.9071
650 iter loss:1.3230 accuracy:0.4090 MAE:0.8900
700 iter loss:1.3096 accuracy:0.4146 MAE:0.8741
750 iter loss:1.2990 accuracy:0.4193 MAE:0.8600
800 iter loss:1.2903 accuracy:0.4248 MAE:0.8484
850 iter loss:1.2829 accuracy:0.4284 MAE:0.8371
900 iter loss:1.2748 accuracy:0.4329 MAE:0.8265
950 iter loss:1.2685 accuracy:0.4368 MAE:0.8188
1000 iter loss:1.2567 accuracy:0.4417 MAE:0.8069
1050 iter loss:1.2485 accuracy:0.4460 MA


epoch:1 total loss:1.1243, accuracy:0.5017, MAE:0.6608, valid_total loss:1.0156, valid_accuracy:0.5466, valid_MAE:0.5716
epoch 2 start!


50 iter loss:0.9091 accuracy:0.6100 MAE:0.4350
100 iter loss:0.8968 accuracy:0.6150 MAE:0.4412
150 iter loss:0.9002 accuracy:0.6142 MAE:0.4408
200 iter loss:0.9021 accuracy:0.6138 MAE:0.4475
250 iter loss:0.8928 accuracy:0.6140 MAE:0.4505
300 iter loss:0.8930 accuracy:0.6125 MAE:0.4500
350 iter loss:0.8885 accuracy:0.6114 MAE:0.4496
400 iter loss:0.8929 accuracy:0.6038 MAE:0.4544
450 iter loss:0.8935 accuracy:0.6019 MAE:0.4550
500 iter loss:0.9001 accuracy:0.6008 MAE:0.4567
550 iter loss:0.8964 accuracy:0.6048 MAE:0.4525
600 iter loss:0.9003 accuracy:0.6023 MAE:0.4550
650 iter loss:0.9005 accuracy:0.6019 MAE:0.4535
700 iter loss:0.9002 accuracy:0.6021 MAE:0.4525
750 iter loss:0.8955 accuracy:0.6042 MAE:0.4497
800 iter loss:0.8936 accuracy:0.6058 MAE:0.4486
850 iter loss:0.8978 accuracy:0.6015 MAE:0.4535
900 iter loss:0.9019 accuracy:0.6007 MAE:0.4574
950 iter loss:0.9020 accuracy:0.6017 MAE:0.4558
1000 iter loss:0.9008 accuracy:0.6026 MAE:0.4550
1050 iter loss:0.9014 accuracy:0.6025 MA


epoch:2 total loss:0.9012, accuracy:0.6050, MAE:0.4556, valid_total loss:1.0036, valid_accuracy:0.5610, valid_MAE:0.5389
epoch 3 start!


50 iter loss:0.7635 accuracy:0.6675 MAE:0.3625
100 iter loss:0.7295 accuracy:0.6887 MAE:0.3325
150 iter loss:0.7199 accuracy:0.6867 MAE:0.3408
200 iter loss:0.7143 accuracy:0.6881 MAE:0.3412
250 iter loss:0.7048 accuracy:0.6950 MAE:0.3325
300 iter loss:0.7125 accuracy:0.6933 MAE:0.3346
350 iter loss:0.7086 accuracy:0.6964 MAE:0.3314
400 iter loss:0.7013 accuracy:0.7000 MAE:0.3278
450 iter loss:0.7059 accuracy:0.6983 MAE:0.3297
500 iter loss:0.7039 accuracy:0.7007 MAE:0.3280
550 iter loss:0.6995 accuracy:0.7039 MAE:0.3245
600 iter loss:0.6970 accuracy:0.7046 MAE:0.3233
650 iter loss:0.6957 accuracy:0.7052 MAE:0.3233
700 iter loss:0.6966 accuracy:0.7055 MAE:0.3225
750 iter loss:0.6938 accuracy:0.7045 MAE:0.3223
800 iter loss:0.6939 accuracy:0.7056 MAE:0.3212
850 iter loss:0.6916 accuracy:0.7076 MAE:0.3188
900 iter loss:0.6959 accuracy:0.7042 MAE:0.3219
950 iter loss:0.6954 accuracy:0.7049 MAE:0.3217
1000 iter loss:0.6932 accuracy:0.7073 MAE:0.3192
1050 iter loss:0.6952 accuracy:0.7061 MA


epoch:3 total loss:0.7079, accuracy:0.7010, MAE:0.3261, valid_total loss:1.1334, valid_accuracy:0.5540, valid_MAE:0.5596
epoch 4 start!


50 iter loss:0.4854 accuracy:0.8100 MAE:0.1925
100 iter loss:0.4747 accuracy:0.8163 MAE:0.1875
150 iter loss:0.4729 accuracy:0.8167 MAE:0.1875
200 iter loss:0.4821 accuracy:0.8131 MAE:0.1913
250 iter loss:0.4955 accuracy:0.8045 MAE:0.2020
300 iter loss:0.4931 accuracy:0.8083 MAE:0.1983
350 iter loss:0.4917 accuracy:0.8068 MAE:0.2000
400 iter loss:0.4930 accuracy:0.8047 MAE:0.2025
450 iter loss:0.4886 accuracy:0.8069 MAE:0.2000
500 iter loss:0.4827 accuracy:0.8093 MAE:0.1973
550 iter loss:0.4921 accuracy:0.8045 MAE:0.2032
600 iter loss:0.4950 accuracy:0.8035 MAE:0.2048
650 iter loss:0.4951 accuracy:0.8042 MAE:0.2044
700 iter loss:0.4964 accuracy:0.8041 MAE:0.2052
750 iter loss:0.4991 accuracy:0.8028 MAE:0.2070
800 iter loss:0.4972 accuracy:0.8033 MAE:0.2061
850 iter loss:0.5001 accuracy:0.8015 MAE:0.2090
900 iter loss:0.5024 accuracy:0.8001 MAE:0.2104
950 iter loss:0.5002 accuracy:0.8012 MAE:0.2099
1000 iter loss:0.4998 accuracy:0.8006 MAE:0.2107
1050 iter loss:0.4979 accuracy:0.8013 MA


epoch:4 total loss:0.5122, accuracy:0.7937, MAE:0.2166, valid_total loss:1.2720, valid_accuracy:0.5499, valid_MAE:0.5483


In [19]:
model.load_state_dict(torch.load(dataset_dir+'model_valid_glsww_allinAsItIs_4epochs.hdf5'))

<All keys matched successfully>

In [20]:
# test datset dataloaderを作成。
test_ds = ReviewDataset(texts=test_df["text"].values)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=1, shuffle=False)

In [21]:
model.eval()
if cuda:
    model.cuda()
else:
    model.cpu()

# 予測を付与。
lists = []
lists_proba = []
for input_ids, segment_ids, attention_masks in tqdm(test_dl):
    if cuda:
        input_ids, segment_ids, attention_masks =\
        input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda()
    outputs = model(input_ids = input_ids,
                  token_type_ids = segment_ids,
                  attention_mask = attention_masks)
    pred_proba = outputs.softmax(dim=1)
    pred = torch.argmax(pred_proba, axis=1)
    lists += list(pred.cpu().detach().numpy()+1)
    lists_proba += list(pred_proba.cpu().detach().numpy()+1)

In [22]:
print(lists)

[5, 2, 1, 2, 4, 2, 4, 3, 3, 3, 2, 1, 2, 2, 4, 2, 1, 4, 1, 3, 4, 4, 3, 3, 5, 2, 4, 5, 5, 5, 5, 1, 5, 1, 4, 2, 2, 1, 4, 3, 3, 5, 5, 4, 3, 2, 5, 1, 4, 2, 4, 5, 2, 5, 3, 2, 2, 4, 3, 1, 1, 5, 2, 2, 3, 5, 5, 1, 2, 3, 3, 2, 4, 3, 2, 4, 1, 3, 2, 5, 2, 2, 1, 2, 4, 3, 3, 5, 4, 3, 2, 3, 1, 2, 2, 2, 4, 2, 2, 2, 5, 4, 4, 3, 5, 3, 2, 4, 5, 4, 5, 2, 3, 5, 5, 2, 5, 2, 3, 4, 2, 4, 4, 4, 5, 3, 3, 2, 4, 5, 4, 1, 5, 4, 3, 1, 3, 3, 5, 1, 5, 1, 2, 1, 3, 5, 1, 1, 5, 1, 3, 1, 3, 5, 5, 3, 5, 4, 3, 1, 3, 1, 5, 3, 2, 2, 5, 2, 2, 1, 5, 4, 1, 2, 2, 2, 4, 3, 1, 3, 4, 4, 1, 1, 5, 4, 2, 4, 2, 1, 4, 5, 4, 1, 5, 2, 3, 4, 1, 1, 5, 2, 4, 1, 3, 4, 2, 4, 5, 4, 1, 3, 3, 2, 4, 3, 1, 3, 1, 3, 5, 2, 3, 5, 2, 4, 5, 4, 2, 2, 3, 1, 3, 1, 1, 1, 3, 5, 4, 3, 4, 2, 5, 3, 5, 4, 2, 4, 2, 1, 4, 3, 5, 4, 5, 3, 3, 3, 2, 4, 5, 2, 2, 2, 2, 4, 4, 2, 3, 5, 1, 4, 1, 2, 1, 1, 1, 4, 3, 3, 4, 3, 4, 2, 3, 1, 2, 2, 5, 4, 2, 1, 1, 4, 3, 3, 4, 5, 2, 4, 3, 2, 2, 5, 5, 4, 3, 4, 1, 5, 1, 5, 4, 1, 4, 4, 4, 3, 5, 5, 1, 4, 5, 3, 2, 1, 4, 3, 4, 5, 1, 2, 2, 

In [23]:
# test_df["Label"]=lists
# submission_df=test_df[["Id","Label"]]
# submission_df.to_csv("sample_submission.csv",index=False)

In [24]:
result = "\n".join([str(rating) for rating in lists])
open(dataset_dir+"test_pred_bert_glsww_allinAsItIs_4epochs.txt", "w").write(result)
result = "\n".join([str(rating) for rating in lists_proba])
open(dataset_dir+"test_predProba_bert_glsww_allinAsItIs_4epochs.txt", "w").write(result)

51999